In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import nltk
import json
import sys
import re
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from dotenv import load_dotenv
from huggingface_hub import login
from nltk.tokenize import sent_tokenize


In [2]:
load_dotenv()
path = os.environ['DATA_PATH']
os.environ["ACCELERATE_USE_TORCH_DEVICE"] = "true"
login(token=os.environ['HF_TOKEN'])
nltk.download('punkt')
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = torch.device("cuda")
print(torch.cuda.get_device_name(0))
print("Supports float16:", torch.cuda.is_available())
print("Supports bfloat16:", torch.cuda.is_bf16_supported())

NVIDIA GeForce RTX 4060 Laptop GPU
Supports float16: True
Supports bfloat16: True


In [ ]:
minervaId = "sapienzanlp/Minerva-1B-base-v1.0"
# minervaId = "sapienzanlp/Minerva-3B-base-v1.0"

tokenizer = AutoTokenizer.from_pretrained(minervaId)
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(minervaId, torch_dtype=torch.float16).to(device)

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\Desktop\Università\MNLP\Homework2\cache\hub\models--sapienzanlp--Minerva-1B-base-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/795k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
datasetRaw = {}

for i in [ocred_fn, cleaned_fn]:
    if i not in os.listdir(path):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(path, i)
    with open(file_path, 'r') as f:
        datasetRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [40]:
sample = datasetRaw['original_ocr']['1']

sentences = sent_tokenize(sample)

print(len(sentences))

adaptor = lambda x: f"""
Sei un esperto di lingua italiana.
Questo testo contiene errori dovuti al fatto che è stato estratto da un immagine.
Correggi il testo mantenendo il più possibile le parole originali.
Non inserire elenchi o numerazioni.
Non aggiungere commenti o testo extra dopo la correzione.
Testo originale: {x}
Testo corretto:
"""
prompts = [adaptor(sentence) for sentence in sentences]
input_tensor = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(device)


40


In [41]:
print(input_tensor["input_ids"])
print("Max token ID:", input_tensor["input_ids"].max().item())
print("Tokenizer vocab size:", tokenizer.vocab_size)

tensor([[    2,     2,     2,  ..., 11388, 32393,    13],
        [    2,     2,     2,  ..., 11388, 32393,    13],
        [    2,     2,     2,  ..., 11388, 32393,    13],
        ...,
        [    2,     2,     2,  ..., 11388, 32393,    13],
        [    2,     2,     2,  ..., 11388, 32393,    13],
        [    2,     2,     2,  ..., 11388, 32393,    13]], device='cuda:0')
Max token ID: 32472
Tokenizer vocab size: 32768


In [54]:
output_tensor = model.generate(
    input_tensor["input_ids"],
    attention_mask=input_tensor["attention_mask"],
    max_new_tokens=512,
    repetition_penalty=1.2,
    do_sample=False,
    num_beams=3,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id 
)

In [55]:
text_extracted = [tokenizer.decode(res, skip_special_tokens=True) for res in output_tensor]
for i, out in enumerate(text_extracted, 1):
    print(f"Output {i}:\n{out}\n{'-'*40}")

Output 1:

Sei un esperto di lingua italiana.
Questo testo contiene errori dovuti al fatto che è stato estratto da un immagine.
Correggi il testo mantenendo il più possibile le parole originali.
Non inserire elenchi o numerazioni.
Non aggiungere commenti o testo extra dopo la correzione.
Testo originale: I. 
Como  andò  che  Maestro  Ciliegia,  Megnamc 
trovò  un  pezzo  di  legno  che  piangeva  e  rideva  come  un  bambino.
Testo corretto:
I.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
----------------------------------------
